In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline
import gc
import os
import logging
import datetime
import warnings
import numpy as np
import pandas as pd
import seaborn as sns
import lightgbm as lgb
from tqdm import tqdm_notebook
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.model_selection import StratifiedKFold
warnings.filterwarnings('ignore')
import time
import random
from tqdm import tqdm_notebook

# Fastai
from fastai import *
from fastai.tabular import *

In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
os.getcwd()

# Any results you write to the current directory are saved as output.
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.metrics import roc_auc_score

import lightgbm as lgb

import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.filterwarnings('ignore')

plt.style.use('seaborn')
sns.set(font_scale=1)

In [26]:
random_state = 42
np.random.seed(random_state)
test_real = pd.read_csv('/storage/santander_comp/real_test.csv')
test_fake = pd.read_csv('/storage/santander_comp/fake_test.csv')

In [29]:
test_real.head()

,ID_code,var_0,var_1,var_2,var_3,var_4,var_5,var_6,var_7,var_8,...,var_190,var_191,var_192,var_193,var_194,var_195,var_196,var_197,var_198,var_199
0,test_3,8.5374,-1.3222,12.0220,6.5749,8.8458,3.1744,4.9397,20.5660,3.3755,...,9.5702,9.0766,1.6580,3.5813,15.1874,3.1656,3.9567,9.2295,13.0168,-4.2108
1,test_7,17.3035,-2.4212,13.3989,8.3998,11.0777,9.6449,5.9596,17.8477,-4.8068,...,4.4676,4.4214,0.9303,1.4994,15.2648,-1.7931,6.5316,10.4855,23.4631,0.7283
2,test_11,10.6137,-2.1898,8.9090,3.8014,13.8602,-5.9802,5.5515,15.4716,-0.1714,...,13.1683,4.0625,-0.1537,7.9787,18.4518,0.1000,-7.8212,9.2355,15.0721,-7.3475
3,test_15,14.8595,-4.5378,13.6483,5.6480,9.9144,1.5190,5.0358,13.4524,-2.5419,...,2.6735,5.8526,4.8517,2.5020,22.8224,-0.9325,8.6849,10.2848,17.4932,6.0800
4,test_16,14.1732,-5.1490,9.7591,3.7316,10.3700,-21.9202,7.7130,18.8749,0.4680,...,0.8640,5.9058,1.3140,4.8961,20.1087,1.1051,7.7184,9.3406,21.1746,-2.0098


In [30]:
test_real['label'] = 'real'

In [31]:
test_fake['label'] = 'fake'

In [39]:
df = test_real.append(test_fake)

In [40]:
# shuffle
df = df.sample(frac=1).reset_index(drop=True)

In [41]:
df.head()

,ID_code,var_0,var_1,var_2,var_3,var_4,var_5,var_6,var_7,var_8,...,var_191,var_192,var_193,var_194,var_195,var_196,var_197,var_198,var_199,label
0,test_193663,5.4328,-5.1310,5.4470,8.1845,11.7997,-6.4869,5.8009,17.7085,-3.5374,...,13.3684,0.6868,7.0986,13.7335,-0.9003,-2.3803,9.4278,10.3598,-14.1094,fake
1,test_86845,13.1111,-1.7106,9.6672,6.9162,13.3853,-6.8097,4.9044,13.8583,3.9771,...,13.6757,0.7782,4.1847,16.2755,-1.2295,1.7018,7.1180,17.2673,13.4447,real
2,test_142212,8.5144,0.7091,8.1496,9.2595,11.2923,5.7124,3.0572,15.7968,-0.0091,...,3.2354,2.5614,-4.3021,22.8922,-0.4521,-0.9529,8.7707,17.1478,3.4314,fake
3,test_170815,11.5884,0.4643,7.7070,4.5403,8.9864,-10.0437,5.0611,20.8001,5.3726,...,14.9746,1.0381,2.0615,15.8277,0.7010,-4.3112,9.1390,11.0018,-13.6520,fake
4,test_20618,14.4554,-5.5968,10.8889,7.5839,13.7141,-7.7395,4.3055,21.3811,-4.8855,...,9.1138,0.8749,-4.6255,18.7480,-0.3094,11.0022,7.9573,18.1593,-13.4505,real


In [60]:
df['label'] = df['label'].astype('category')
df['label_codes'] = df['label'].cat.codes

In [61]:
df.head()

,ID_code,var_0,var_1,var_2,var_3,var_4,var_5,var_6,var_7,var_8,...,var_192,var_193,var_194,var_195,var_196,var_197,var_198,var_199,label,label_codes
0,test_193663,5.4328,-5.1310,5.4470,8.1845,11.7997,-6.4869,5.8009,17.7085,-3.5374,...,0.6868,7.0986,13.7335,-0.9003,-2.3803,9.4278,10.3598,-14.1094,fake,0
1,test_86845,13.1111,-1.7106,9.6672,6.9162,13.3853,-6.8097,4.9044,13.8583,3.9771,...,0.7782,4.1847,16.2755,-1.2295,1.7018,7.1180,17.2673,13.4447,real,1
2,test_142212,8.5144,0.7091,8.1496,9.2595,11.2923,5.7124,3.0572,15.7968,-0.0091,...,2.5614,-4.3021,22.8922,-0.4521,-0.9529,8.7707,17.1478,3.4314,fake,0
3,test_170815,11.5884,0.4643,7.7070,4.5403,8.9864,-10.0437,5.0611,20.8001,5.3726,...,1.0381,2.0615,15.8277,0.7010,-4.3112,9.1390,11.0018,-13.6520,fake,0
4,test_20618,14.4554,-5.5968,10.8889,7.5839,13.7141,-7.7395,4.3055,21.3811,-4.8855,...,0.8749,-4.6255,18.7480,-0.3094,11.0022,7.9573,18.1593,-13.4505,real,1


In [42]:
lgb_params = {
    "objective" : "binary",
    "metric" : "auc",
    "boosting": 'gbdt',
    "max_depth" : -1,
    "num_leaves" : 13,
    "num_threads": 8,
    "learning_rate" : 0.0085,
    "bagging_freq": 5,
    "bagging_fraction" : 0.4,
    "feature_fraction" : 0.04,
    "min_data_in_leaf": 80,
    "min_sum_heassian_in_leaf": 10,
    "tree_learner": "serial",
    "boost_from_average": "false",
    #"lambda_l1" : 5,
    #"lambda_l2" : 5,
    "bagging_seed" : random_state,
    "verbosity" : -1,
    "seed": random_state
}

In [43]:
X = df.copy()


In [45]:
X.drop(['ID_code', 'label'], axis = 1, inplace=True)

In [62]:
y = df['label_codes']

In [63]:
from sklearn.model_selection import train_test_split

In [64]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size = 0.2, random_state = 2019)

In [65]:
trn_data = lgb.Dataset(X_train, label = y_train)
val_data = lgb.Dataset(X_val, label = y_val)

In [66]:
evals_result = {}

In [68]:
lgb_clf = lgb.train(lgb_params,
                        trn_data,
                        100000,
                        valid_sets = [trn_data, val_data],
                        early_stopping_rounds=5000,
                        verbose_eval = 1000,
                        evals_result=evals_result
                       )

Training until validation scores don't improve for 5000 rounds.
[1000]	training's auc: 0.699005	valid_1's auc: 0.474541
[2000]	training's auc: 0.74048	valid_1's auc: 0.469266
[3000]	training's auc: 0.768804	valid_1's auc: 0.468204
[4000]	training's auc: 0.789756	valid_1's auc: 0.466676
[5000]	training's auc: 0.810027	valid_1's auc: 0.464841
Early stopping, best iteration is:
[10]	training's auc: 0.523961	valid_1's auc: 0.500125
